# Building A Chatbot
An example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This notebook will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1214193d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x12141ad10>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="What is machine learning?")])

AIMessage(content='Machine learning is a type of artificial intelligence (AI) that allows computers to learn from data without being explicitly programmed. \n\nHere\'s a breakdown:\n\n**Instead of:**\n\n* **Being told exactly what to do** for every situation (like a traditional program),\n\n**Machine learning algorithms:**\n\n* **Identify patterns and relationships** in large datasets.\n* **Use those patterns to make predictions or decisions** about new, unseen data.\n\n**Think of it like this:**\n\nImagine teaching a child to recognize a cat. You wouldn\'t write a list of all the features that make a cat a cat (furry, four legs, whiskers, etc.). Instead, you\'d show them lots of pictures of cats and say "This is a cat." Over time, the child learns to recognize the patterns that define a cat and can identify new cats they haven\'t seen before.\n\nMachine learning works in a similar way.\n\n**Types of Machine Learning:**\n\n* **Supervised Learning:** The algorithm is trained on labeled 

In [4]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="What is machine learning?"),
    AIMessage(content="Machine learning is a type of artificial intelligence (AI) that allows computers to learn from data without being explicitly programmed."),
    HumanMessage(content="What's the topic being dicussed now?")
])

AIMessage(content="The topic we are discussing now is **machine learning**.  \n\nI explained what it is, and you asked me to elaborate.  \n\nIs there anything else you'd like to know about machine learning?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 58, 'total_tokens': 103, 'completion_time': 0.081818182, 'prompt_time': 0.001736614, 'queue_time': 0.012193514999999999, 'total_time': 0.083554796}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-40e327b9-cda7-438c-913b-b8f6d8550a8a-0', usage_metadata={'input_tokens': 58, 'output_tokens': 45, 'total_tokens': 103})

## Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable": {"session_id": "chat1"}}

response = with_message_history.invoke([HumanMessage(content="What is machine learning?")], config=config)
response.content

'Machine learning is a type of artificial intelligence (AI) that allows computers to learn from data without being explicitly programmed. \n\nHere\'s a breakdown:\n\n**Instead of giving a computer specific instructions for every possible scenario, you feed it large amounts of data.** The machine then uses algorithms to identify patterns, relationships, and insights within that data. This allows it to make predictions or decisions on new, unseen data.\n\n**Think of it like this:**\n\n* **Traditional programming:** You write a recipe (code) that tells the computer exactly how to make a cake.\n* **Machine learning:** You give the computer many pictures of cakes and tell it which ones are good and which ones are bad. The computer learns the features of a good cake and can then identify good cakes in new pictures.\n\n**Types of Machine Learning:**\n\n* **Supervised learning:** The algorithm is trained on labeled data (e.g., images labeled as "cat" or "dog").\n* **Unsupervised learning:** Th

In [8]:
with_message_history.invoke([HumanMessage(content="What's the topic being dicussed now in this chat?")], config=config)

AIMessage(content="The topic we are discussing now is **machine learning**.  \n\nYou asked me to define it, and we've been going back and forth about its different types and examples.  😊 \n\n\nWhat else would you like to know about machine learning?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 552, 'total_tokens': 606, 'completion_time': 0.098181818, 'prompt_time': 0.019663469, 'queue_time': 0.092835181, 'total_time': 0.117845287}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fd7de776-0918-435c-88ff-28ab5424adff-0', usage_metadata={'input_tokens': 552, 'output_tokens': 54, 'total_tokens': 606})

In [9]:
## change the config-->session id
config1 = {"configurable": {"session_id": "chat2"}}

response = with_message_history.invoke([HumanMessage(content="What's the topic being dicussed now in this chat?")], config=config1)
response.content

"As a large language model, I don't have memory of past conversations. Every interaction we have is a fresh start.\n\nWhat topic would you like to discuss? 😊\n"

## Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [11]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the question to the best of your ability."),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt | model

In [12]:
chain.invoke({"messages": [HumanMessage(content="What is machine learning?")]})

AIMessage(content="Machine learning is a type of artificial intelligence (AI) that allows computers to learn from data without being explicitly programmed. \n\nHere's a breakdown:\n\n**Instead of giving a computer specific instructions for every possible scenario, machine learning algorithms are trained on large datasets.**  They identify patterns, relationships, and insights within the data, and use those learnings to make predictions or decisions on new, unseen data.\n\n**Think of it like this:**\n\n* **Traditional programming:** You tell the computer exactly what to do step-by-step.\n* **Machine learning:** You give the computer data and let it figure out the rules and patterns on its own.\n\n**Types of Machine Learning:**\n\n* **Supervised learning:** The algorithm is trained on labeled data (data with known inputs and outputs). It learns to map inputs to outputs, allowing it to make predictions on new data.\n* **Unsupervised learning:** The algorithm is trained on unlabeled data a

In [13]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [14]:
config = {"configurable": {"session_id": "chat3"}}

response = with_message_history.invoke([HumanMessage(content="What is machine learning?")], config=config)
response

AIMessage(content='Machine learning is a type of artificial intelligence (AI) that allows computers to learn from data without being explicitly programmed. \n\nHere\'s a breakdown:\n\n**Instead of giving computers specific instructions for every scenario, we feed them large amounts of data and let them identify patterns and relationships.**  \n\nThink of it like teaching a child to recognize a cat. You wouldn\'t write a program with every possible feature of a cat (whiskers, pointy ears, furry, etc.). Instead, you\'d show them many pictures of cats, and they\'d learn to identify the common characteristics.\n\n**Machine learning algorithms** are the "students" in this scenario. They analyze the data, adjust their internal parameters, and improve their ability to make predictions or decisions based on new, unseen data.\n\n**There are different types of machine learning:**\n\n* **Supervised learning:** The algorithm is trained on labeled data (e.g., images labeled "cat" or "not cat"). It 

In [15]:
response = with_message_history.invoke([HumanMessage(content="What's the topic being dicussed now in this chat?")], config=config)
response.content

"The topic we're discussing is **machine learning**.  \n\nI explained what it is, the different types, and some of its applications. \n\nDo you have any more questions about it? 😊  \n"

In [16]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."), 
    MessagesPlaceholder(variable_name="messages")
])
chain = prompt | model

In [18]:
response = chain.invoke({"messages": [HumanMessage(content="What's the largest city in the world based on the area?")], "language": "Tamil"})
response.content

'புறநிலைப் பரப்பின் அடிப்படையில் உலகின் மிகப்பெரிய நகரம் **டோக்கியோ** ஆகும். \n'

In [19]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [20]:
config = {"configurable": {"session_id": "chat4"}}
repsonse = with_message_history.invoke(
    {'messages': [HumanMessage(content="John here, how are you?")], "language": "Tamil"},
    config=config
)
repsonse.content

'வணக்கம் ஜான்! நான் நன்றாக இருக்கிறேன். நீ எப்படி இருக்கிறாய்? 😊 \n'

In [21]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What's my name?")], "language": "Tamil"},
    config=config,
)
response

AIMessage(content='உங்கள் பெயர் ஜான்.  😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 79, 'total_tokens': 93, 'completion_time': 0.025454545, 'prompt_time': 0.002550381, 'queue_time': 0.012162926000000001, 'total_time': 0.028004926}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bc70fc61-2ddc-4de7-bcea-87122ed7032e-0', usage_metadata={'input_tokens': 79, 'output_tokens': 14, 'total_tokens': 93})

## Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [22]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/Users/kamaraj/scm/ai/genai-langchain/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kamaraj/scm/ai/genai-langchain/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's 2 + 2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [23]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke({
    "messages":messages + [HumanMessage(content="What ice cream do I like?")],
    "language":"English"
})
response.content

"As a helpful assistant, I don't have personal preferences or memories, so I don't know what ice cream you like! \n\nWhat's your favorite flavor? 😄🍦  \n"

In [24]:
response = chain.invoke({
    "messages": messages + [HumanMessage(content="what math problem did I ask?")],
    "language": "English",
})
response.content

'You asked "What is 2 + 2?"  \n'